In [1]:
import pandas as pd

df=pd.read_csv('final_plotly_data.csv')

In [6]:
df.dtypes

date            object
query           object
page            object
device          object
country         object
clicks           int64
impressions      int64
ctr            float64
position       float64
blog_link       object
type            object
dtype: object

In [12]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Aggregate data by type
type_metrics = df.groupby('type').agg({
    'clicks': 'sum',
    'impressions': 'sum',
    'ctr': 'mean'
}).reset_index()

# Create subplots with 3 rows
fig = make_subplots(rows=3, cols=1,
                    subplot_titles=('Clicks by Type', 'Impressions by Type', 'Average CTR by Type'),
                    vertical_spacing=0.1)

# Add bar chart for Clicks
fig.add_trace(
    go.Bar(x=type_metrics['type'], y=type_metrics['clicks'], name='Clicks'),
    row=1, col=1
)

# Add bar chart for Impressions
fig.add_trace(
    go.Bar(x=type_metrics['type'], y=type_metrics['impressions'], name='Impressions'),
    row=2, col=1
)

# Add bar chart for CTR
fig.add_trace(
    go.Bar(x=type_metrics['type'], y=type_metrics['ctr'], name='CTR'),
    row=3, col=1
)

# Update layout
fig.update_layout(
    height=900,  # Increase height to accommodate three graphs
    showlegend=False,
    title_text="Performance Metrics by Type",
    title_x=0.5
)

# Update y-axes labels
fig.update_yaxes(title_text="Clicks", row=1, col=1)
fig.update_yaxes(title_text="Impressions", row=2, col=1)
fig.update_yaxes(title_text="CTR", row=3, col=1)

# Show the figure
fig.show()

# Alternative: Create separate graphs

# Clicks
fig_clicks = px.bar(type_metrics, x='type', y='clicks',
                   title='Clicks by Type',
                   labels={'clicks': 'Total Clicks', 'type': 'Type'})

# Impressions
fig_impressions = px.bar(type_metrics, x='type', y='impressions',
                        title='Impressions by Type',
                        labels={'impressions': 'Total Impressions', 'type': 'Type'})

# CTR
fig_ctr = px.bar(type_metrics, x='type', y='ctr',
                 title='Average CTR by Type',
                 labels={'ctr': 'Average CTR', 'type': 'Type'})

# Show individual figures
fig_clicks.show()
fig_impressions.show()
fig_ctr.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [13]:
!pip install nbformat>=4.2.0

zsh:1: 4.2.0 not found


In [18]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Convert date to datetime if it's not already
df['date'] = pd.to_datetime(df['date'])

# Per-type metrics
type_metrics = df.groupby('type').agg({
    'clicks': 'sum',
    'impressions': 'sum',
    'ctr': 'mean',
    'page': 'nunique'
}).reset_index()

# Calculate per-page metrics
type_metrics['clicks_per_page'] = type_metrics['clicks'] / type_metrics['page']
type_metrics['impressions_per_page'] = type_metrics['impressions'] / type_metrics['page']
type_metrics['ctr_per_page'] = type_metrics['ctr']

# Time-based metrics
time_metrics = df.groupby(['date', 'type']).agg({
    'clicks': 'sum',
    'impressions': 'sum',
    'ctr': 'mean'
}).reset_index()

# Create subplots with 6 rows
fig = make_subplots(
    rows=6, 
    cols=1,
    subplot_titles=(
        'Average Clicks per Page by Type', 
        'Average Impressions per Page by Type', 
        'Average CTR by Type',
        'Clicks Over Time by Type',
        'Impressions Over Time by Type',
        'CTR Over Time by Type'
    ),
    vertical_spacing=0.05
)

# Add traces for per-page metrics
# 1. Clicks per page
fig.add_trace(
    go.Bar(x=type_metrics['type'], 
           y=type_metrics['clicks_per_page'], 
           name='Clicks per Page',
           hovertemplate="Type: %{x}<br>Clicks per Page: %{y:.1f}<extra></extra>"),
    row=1, col=1
)

# 2. Impressions per page
fig.add_trace(
    go.Bar(x=type_metrics['type'], 
           y=type_metrics['impressions_per_page'], 
           name='Impressions per Page',
           hovertemplate="Type: %{x}<br>Impressions per Page: %{y:.1f}<extra></extra>"),
    row=2, col=1
)

# 3. CTR
fig.add_trace(
    go.Bar(x=type_metrics['type'], 
           y=type_metrics['ctr_per_page'], 
           name='CTR',
           hovertemplate="Type: %{x}<br>CTR: %{y:.2%}<extra></extra>"),
    row=3, col=1
)

# Add time series traces
# 4. Clicks over time
for type_name in df['type'].unique():
    type_data = time_metrics[time_metrics['type'] == type_name]
    fig.add_trace(
        go.Scatter(x=type_data['date'], 
                  y=type_data['clicks'],
                  name=f'{type_name} Clicks',
                  mode='lines',
                  hovertemplate="Date: %{x}<br>Clicks: %{y:.0f}<extra></extra>"),
        row=4, col=1
    )

# 5. Impressions over time
for type_name in df['type'].unique():
    type_data = time_metrics[time_metrics['type'] == type_name]
    fig.add_trace(
        go.Scatter(x=type_data['date'], 
                  y=type_data['impressions'],
                  name=f'{type_name} Impressions',
                  mode='lines',
                  hovertemplate="Date: %{x}<br>Impressions: %{y:.0f}<extra></extra>"),
        row=5, col=1
    )

# 6. CTR over time
for type_name in df['type'].unique():
    type_data = time_metrics[time_metrics['type'] == type_name]
    fig.add_trace(
        go.Scatter(x=type_data['date'], 
                  y=type_data['ctr'],
                  name=f'{type_name} CTR',
                  mode='lines',
                  hovertemplate="Date: %{x}<br>CTR: %{y:.2%}<extra></extra>"),
        row=6, col=1
    )

# Update layout
fig.update_layout(
    height=1800,  # Set height here
    title_text="Search Console Performance Metrics Analysis",
    title_x=0.5,
    plot_bgcolor='white',
    bargap=0.2,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    )
)

# Update axes labels and styling
fig.update_yaxes(title_text="Clicks per Page", row=1, col=1, gridcolor='lightgrey')
fig.update_yaxes(title_text="Impressions per Page", row=2, col=1, gridcolor='lightgrey')
fig.update_yaxes(title_text="CTR", row=3, col=1, gridcolor='lightgrey')
fig.update_yaxes(title_text="Clicks", row=4, col=1, gridcolor='lightgrey')
fig.update_yaxes(title_text="Impressions", row=5, col=1, gridcolor='lightgrey')
fig.update_yaxes(title_text="CTR", row=6, col=1, gridcolor='lightgrey')

# Update x-axes
for i in range(1, 7):
    fig.update_xaxes(gridcolor='lightgrey', row=i, col=1)

# Add different colors for bar charts
fig.update_traces(marker_color='rgb(158,202,225)', row=1, col=1)
fig.update_traces(marker_color='rgb(94,158,217)', row=2, col=1)
fig.update_traces(marker_color='rgb(32,102,148)', row=3, col=1)

# Save to single HTML file
fig.write_html("search_console_metrics_complete.html")

# Print summary statistics
print("\nPer-Page Summary Statistics:")
summary_df = type_metrics[['type', 'clicks_per_page', 'impressions_per_page', 'ctr_per_page', 'page']]
print(summary_df.round(2))

# Time-based summary statistics
print("\nTime-based Summary Statistics:")
time_summary = time_metrics.groupby('type').agg({
    'clicks': ['mean', 'min', 'max'],
    'impressions': ['mean', 'min', 'max'],
    'ctr': ['mean', 'min', 'max']
}).round(2)
print(time_summary)


Per-Page Summary Statistics:
     type  clicks_per_page  impressions_per_page  ctr_per_page  page
0      AI             1.44                817.60          0.00    25
1  manual            23.59               1200.65          0.02    74

Time-based Summary Statistics:
       clicks         impressions              ctr           
         mean min max        mean  min   max  mean  min   max
type                                                         
AI       0.43   0   4      243.33    1  1829  0.01  0.0  0.38
manual  19.62   3  48      998.29  415  1843  0.02  0.0  0.03
